# Readability Scores

There are many readability score available. Some of the them are as follows -
<ol>
    <li> Flesch-Kincaid: This score is useful when determining the grade level needed to understand a piece of text. It takes into account sentence length and word complexity, and provides a grade-level score. It can be useful for educational materials or wehn targeting a specific reading level.</li>
    <li>Coleman-Liau: This score also provides a grade-level score but places greater emphasis on the length of the text and the number of characters per word. It can be useful for determining the reading level of technical or scientific writing.</li>
    <li>Dale-Chall: This score is useful for identifying difficult words that may be challenging for readers. It uses a list of common words that are easy to understand and compares them to the text. This score is useful when creating materials for readers with limited vocabulary.</li>
    <li>SMOG: This score is similar to the Flesch-Kincaid score, but it places greater emphasis on longer words. It can be useful for determining the reading level of technical or scientific writing.</li>
    <li>Automated Readability Index: This score is useful for determining the reading level of general writing. It takes into account the length of the text and the number of words per sentence.</li>
    <li>Flesch Reading Ease: This score is useful for determining the readability of general writing. It provides a score between 0 and 100, with higher scores indicating easier readability. </li>
    <li>The Gunning fog formula: This score is useful for determining the reading level of general writing. It takes into account sentence length and the number of complex words used. </li>
    <li>Fry readability graph: This score is useful for determining the reading level of children's books. It provides a graph that shows the readability level of the text.</li>
    <li>The FORECAST formula: This score is useful for determining the reading level of technical or scientific writing. It takes into account sentence length, word complexity, and the number of syllables per word.</li>
</ol>

From this we can conclude that to check the readability score for a technical document such as Terms of Service, the following scores can be used -
<li>Flesch-Kincaid</li>
<li>Automated Readability Index</li>
<li>The Gunning fog formula</li>

We are using Flesch-Kincaid Readability Score.

In [1]:
import spacy
from textstat.textstat import textstatistics
from textstat import textstat
import pandas as pd

Generral steps to apply the formula:-
<ul>
    <li>Select several 100-word samples throughout the text. </li>
    <li>Compute the average sentence length in words (divide the number of words by the number of sentences).</li> 
    <li>Compute the percentage of words NOT on the Dale–Chall word list of 3, 000 easy words.</li> 
    <li>Compute the respective equations</li> 
</ul> 

In [2]:
def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)

In [3]:
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words

In [4]:
def sentence_count(text):
    sentences = break_sentences(text)
    return len(sentences)

In [5]:
def avg_sentence_length(text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length

In [6]:
def syllables_count(word):
    return textstatistics().syllable_count(word)

In [7]:
def difficult_words(text):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    # difficult words are those with syllables >= 2 
    # easy_word_set is provide by Textstat as a list of common words
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)

In [8]:
# A word is polysyllablic if it has more than 3 syllables. 
def poly_syllable_count(text):
    count = 0
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [token for token in sentence]
     
 
    for word in words:
        syllable_count = syllables_count(word)
        if syllable_count >= 3:
            count += 1
    return count

### Flesch-Kincaid Grade Level
The U.S. Army uses Flesch-Kincaid Grade Level for assessing the difficulty of technical manuals. The commonwealth of Pennsylvania uses Flesch-Kincaid Grade Level for scoring automobile insurance policies to ensure their texts are no higher than a ninth grade level of reading difficulty. Many other U.S. states also use Flesch-Kincaid Grade Level to score other legal documents such as business policies and financial forms.

In [14]:
def flesch_kincaid(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_syllables = syllables_count(text)
    score =  0.39 * (num_words / num_sentences) + 11.8 * (num_syllables / num_words) - 15.59
    return score

In [15]:
text1 = "account termination policy youtube will terminate a user s access to the service if under appropriate circumstances the user is determined to be a repeat infringer. youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length. youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service."
text2 = "if you infringe copyright multiple times we close your account. if you are in violation of our community guidelines we may do that immediately."
score_text1 = flesch_kincaid(text1)
score_text2 = flesch_kincaid(text2)
print(score_text1, score_text2)

15.786021505376347 7.633846153846157


In [11]:
def ARI(text):
    num_chars = len(text)
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    score = 4.71 * (num_chars / num_words) + 0.5 * (num_words / num_chars) - 21.43
    return score

In [12]:
text1 = "account termination policy youtube will terminate a user s access to the service if under appropriate circumstances the user is determined to be a repeat infringer. youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length. youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service."
text2 = "if you infringe copyright multiple times we close your account. if you are in violation of our community guidelines we may do that immediately."
score_text1 = ARI(text1)
score_text2 = ARI(text2)
print(score_text1, score_text2)

7.014326036866358 4.565909090909091


In [16]:
def gunning_fog(text):
    num_words = word_count(text)
    num_sentences = sentence_count(text)
    num_complex_words = difficult_words(text)
    score = 0.4 * ((num_words / num_sentences) + 100 * (num_complex_words / num_words))
    return score

In [17]:
text1 = "account termination policy youtube will terminate a user s access to the service if under appropriate circumstances the user is determined to be a repeat infringer. youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length. youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service."
text2 = "if you infringe copyright multiple times we close your account. if you are in violation of our community guidelines we may do that immediately."
score_text1 = gunning_fog(text1)
score_text2 = gunning_fog(text2)
print(score_text1, score_text2)

25.303225806451614 17.50769230769231


In [20]:
df = pd.read_csv('Dataset/all_v1_transpose.csv')
df.head()

,doc,id,original_text,reference_summary,title,uid,case_code,case_text,note,title_code,title_text,urls,tldr_code,tldr_text
0,Pokemon GO Terms of Service,5786730a6cca83a54c0035b7,welcome to the pokémon go video game services ...,hi.,NaN,legalsum01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pokemon GO Terms of Service,57866df76cca83a54c0035a1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....,Agreement To Terms,legalsum02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pokemon GO Terms of Service,5786730a6cca83a54c0035b6,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...,Eligibility and Account Registration,legalsum03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pokemon GO Terms of Service,57866df76cca83a54c0035a0,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...,Safe Play,legalsum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pokemon GO Terms of Service,57866df76cca83a54c00359f,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...,Rights in App,legalsum05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
to_drop = ['id', 'doc', 'title', 'uid', 'case_code', 'case_text', 'note', 'title_code', 'title_text', 'urls', 'tldr_code', 'tldr_text']
df.drop(to_drop, axis = 1, inplace = True)
df.head()

,original_text,reference_summary
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [ ]:
df['Original Text Score'] = df['original_text'].apply(flesch_kincaid)
df['Reference Summary Score'] = df['reference_summary'].apply(flesch_kincaid)